In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False
if fls.is_submission:
    fast_mode = False

MainProcess


In [2]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:30]
    test_data = test_data[2:4]
len(train_data), len(test_data)

extra_data = fls.load_all_extra_data(for_yolo=True)
np.random.default_rng(seed=0).shuffle(extra_data)
extra_data = extra_data[:300]
if fast_mode: extra_data = extra_data[:5]

In [3]:
import pandas as pd
def get_model_for_seed(overall_seed):
    model = flg_yolo.YOLOModel()
    model.seed = overall_seed
    model.use_augs = True
    model.auto_augment = None
    rng = np.random.default_rng(seed=overall_seed)
    if rng.uniform()<0.5: model.hsv_h = 0.
    if rng.uniform()<0.5: model.hsv_s = 0.
    if rng.uniform()<0.5: model.hsv_v = 0.
    if rng.uniform()<0.5: model.translate = 0.
    if rng.uniform()<0.5: model.scale = 0.
    if rng.uniform()<0.5: model.fliplr = 0.
    if rng.uniform()<0.5: model.mosaic = 0.
    if rng.uniform()<0.5: model.mixup = 0.
    if rng.uniform()<0.5: model.erasing = 0.
    model.run_in_parallel = True
    if fast_mode: model.n_epochs = 1
    return model

for seed in [5,6,7,8,9,10,11,12,13]:
    model = get_model_for_seed(seed)
    model.train(train_data, test_data)
    fls.dill_save(fls.model_dir + 'rand_model_' + str(seed) + '.pickle', model)
    test_data_i = model.infer(test_data)
    r_test = fls.score_competition_metric(test_data_i, test_data)
    print(r_test)
    extra_data_i = model.infer(extra_data)
    r_extra = fls.score_competition_metric(extra_data_i, extra_data)    
    print(r_extra)
    fls.dill_save(fls.model_dir + 'res_' + str(seed) + '.pickle', (r_test, r_extra, test_data_i, test_data, extra_data_i, extra_data))

5
['tomo_507b7a', 'tomo_b33d4e', 'tomo_b7d014', 'tomo_b0ded6', 'tomo_37dd38']
['tomo_2dcd5c', 'tomo_acadd7', 'tomo_634b06', 'tomo_b2ebbc', 'tomo_1f0e78']
Will process approximately 3123 slices for training


Processing training motors:   0%|          | 0/347 [00:00<?, ?it/s]

Will process approximately 900 slices for validation


Processing validation motors:   0%|          | 0/100 [00:00<?, ?it/s]


Processing Summary:
- Train set: 258 tomograms, 347 motors, 3118 slices
- Validation set: 100 tomograms, 100 motors, 900 slices
- Total: 358 tomograms, 447 motors, 4018 slices

Preprocessing Complete:
- Training data: 258 tomograms, 347 motors, 3118 slices
- Validation data: 100 tomograms, 100 motors, 900 slices
- Dataset directory: d:/flagellar/temp//yolo_dataset/
- YAML configuration: d:/flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Directory status:
- Train images exists: True
- Val images exists: True
- Train labels exists: True
- Val labels exists: True
Found original dataset.yaml at d:/flagellar/temp//yolo_dataset/dataset.yaml
Fixing YAML paths in d:/flagellar/temp//yolo_dataset/dataset.yaml
Created fixed YAML at d:/flagellar/temp/fixed_dataset.yaml with path: d:/flagellar/temp//yolo_dataset/
Using YAML file: d:/flagellar/temp/fixed_dataset.yaml
YAML contents:
names:
  0: motor
path: d:/flagellar/temp//yolo_dataset/
train: 

...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07

                   all        900        900      0.654      0.652      0.651      0.227



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



      4/30      8.28G       2.23      1.846      1.026         16        640:   3%|▎         | 6/195 [00:01<00:49,  3.

KeyboardInterrupt: 

In [3]:
model = fls.dill_load(fls.model_dir + '/unet_normbugfixed.pickle')
test_data_i = model.infer(test_data)
r_test = fls.score_competition_metric(test_data_i, test_data)
print(r_test)
extra_data_i = model.infer(extra_data)
r_extra = fls.score_competition_metric(extra_data_i, extra_data)    
print(r_extra)
fls.dill_save(fls.model_dir + 'res_unet_normbugfixed.pickle', (r_test, r_extra, test_data_i, test_data, extra_data_i, extra_data))

0.8165322580645161
0.04541701073492981


In [ ]:
model = fls.dill_load(fls.model_dir + '/unet_extradata.pickle')
test_data_i = model.infer(test_data)
r_test = fls.score_competition_metric(test_data_i, test_data)
print(r_test)
extra_data_i = model.infer(extra_data)
r_extra = fls.score_competition_metric(extra_data_i, extra_data)    
print(r_extra)
fls.dill_save(fls.model_dir + 'res_unet_extradata.pickle', (r_test, r_extra, test_data_i, test_data, extra_data_i, extra_data))

In [ ]:
for seed in [0,1,2,3,4]:
    model = fls.dill_load(fls.model_dir + '/rand_model_' + str(seed) + '.pickle')
    extra_data_i = model.infer(extra_data)
    r_extra = fls.score_competition_metric(extra_data_i, extra_data)    
    print(seed, r_extra)
    #fls.dill_save(fls.model_dir + 'res_unet_normbugfixed.pickle', (r_test, r_extra, test_data_i, test_data, extra_data_i, extra_data))

0 0.38461538461538464
1 0.2644041041831097
2 0.4337899543378995
